### Install Dependencies

In [1]:
%pip install -q langchain
%pip install -q langchain-ibm
%pip install -q langchain-community
%pip install -q ibm-watsonx-ai
%pip install -q ibm-watson-machine_learning
%pip install -q chromadb
%pip install -q tiktoken
%pip install -q beautifulsoup4
%pip install -q python-dotenv



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is a

In [3]:
!brew install poppler tesseract libmagic

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
dockerfmt                                fedify
==> New Casks
antinote                   dante-controller           font-libertinus-mono
companion                  dante-via                  ijhttp
companion-satellite        excire-foto                ndi-tools
companion@beta             font-exile                 ovice

You have 17 outdated formulae installed.

To reinstall 5.5.0_1, run:
  brew reinstall tesseract
==> Downloading https://ghcr.io/v2/homebrew/core/poppler/manifests/25.04.0
######################################################################### 100.0%
==> Fetching dependencies for poppler: glib, libidn2, gnutls, libgpg-error, libassuan, libgcrypt, libksba, libusb, npth, pinentry, gnupg, 

### Import Libraries

In [4]:
#Import libraries

import os

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
# from dotenv import load_dotenv



USER_AGENT environment variable not set, consider setting it to identify your requests.


### Initialize Models

In [ ]:
#Set up keys and api endpoints

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "XHfmHfEIismZnFOi-vviiM-IHbxwnbsaFbM0DA12qhuT",
    "model" : "ibm/granite-3-2-8b-instruct",
    "project_id": "02464812-b861-454c-b58c-2cb85043d848",
}

In [6]:
# Load in model and parameters

llm = WatsonxLLM(
    model_id=credentials["model"],
    url =credentials["url"],
    apikey=credentials["apikey"],
    project_id=credentials["project_id"],
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 250,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"]
    },   
)
    

Model 'meta-llama/llama-4-maverick-17b-128e-instruct-fp' is not supported for this environment. Supported models: ['google/flan-t5-xl', 'google/flan-t5-xxl', 'google/flan-ul2', 'ibm/granite-13b-instruct-v2', 'ibm/granite-20b-code-instruct', 'ibm/granite-20b-multilingual', 'ibm/granite-3-2-8b-instruct', 'ibm/granite-3-2b-instruct', 'ibm/granite-3-8b-instruct', 'ibm/granite-34b-code-instruct', 'ibm/granite-3b-code-instruct', 'ibm/granite-8b-code-instruct', 'ibm/granite-guardian-3-2b', 'ibm/granite-guardian-3-8b', 'ibm/granite-vision-3-2-2b', 'meta-llama/llama-2-13b-chat', 'meta-llama/llama-3-1-70b-instruct', 'meta-llama/llama-3-1-8b-instruct', 'meta-llama/llama-3-2-11b-vision-instruct', 'meta-llama/llama-3-2-1b-instruct', 'meta-llama/llama-3-2-3b-instruct', 'meta-llama/llama-3-2-90b-vision-instruct', 'meta-llama/llama-3-3-70b-instruct', 'meta-llama/llama-3-405b-instruct', 'meta-llama/llama-4-maverick-17b-128e-instruct-fp8', 'meta-llama/llama-4-scout-17b-16e-instruct', 'meta-llama/llama-guard-3-11b-vision', 'mistralai/mistral-large', 'mistralai/mixtral-8x7b-instruct-v01']


In [7]:
# Set up prompt template

template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

In [8]:
# Set up a chain with our prompt and LLM

agent = prompt | llm  #not sure how this works 

In [9]:
#basic query

agent.invoke({"query": "What sport is played at the US Open?"}) # has general data about tennis and us open but super speceific and gets correct answer

' Do not try to create plausible-sounding but false answers.\n\nThe sport played at the US Open is Tennis.'

In [10]:
agent.invoke({"query": "What is the capital of France?"}) #also generic question that should have data for

'\n\nThe capital of France is Paris.'

In [11]:
agent.invoke({"query": "Where was the 2024 US Open Tennis Championship?"}) # does not have data for 2024 so it says "I do not know" which is correct.

' Do not invent an answer.\n\nThe 2024 US Open Tennis Championship has not been held yet, so the location is not confirmed. Please check back closer to the event for the most accurate information.'

In [ ]:
agent.invoke({"query": "Where was the 2026 US Open Tennis Championship?"}) # does not have data for 2026 so it says "I do not know" which is correct.

' Do not invent an answer.\n\nThe 2026 US Open Tennis Championships have not been officially announced yet, so the location is not confirmed. Therefore, I cannot provide the accurate location for the 2026 US Open Tennis Championship.'

### Load Data

In [18]:
%pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of local PDF paths
pdf_paths = [
    "doc 1.pdf",
    "doc 2.pdf",
    "doc 3.pdf",
    "doc 4.pdf",
]

# Load PDFs


docs = [UnstructuredPDFLoader(path).load() for path in pdf_paths]
docs_list = [item for sublist in docs for item in sublist]  # Flatten
docs_list[0]  # Check the first document

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


Document(metadata={'source': 'doc 1.pdf'}, page_content='4/9/25, 8:57 PM U.S. Open | IBM\n\nHome / Case Studies / US Open\n\nAcing the US Open d experience\n\nAI models built with watsonx transfor data into insight\n\nGet the latest Aland techinsights —>\n\nLearn More > |\n\nhttps://www.ibm.com/case-studies/us-open 1/12\n\n4/9/25, 8:57 PM U.S. Open | IBM\n\nBusiness Challenge Transformation Out <\n\nFor two weeks at the end of summer, nearly one Flushing, New York, to watch the best tennis pl: Open Tennis Championships.\n\nYear after year, it is one of the most highly atten\n\nBut more than 15 million global tennis fans follc Open app and website. And to keep them comir Tennis Association (USTA) has worked side-by-: than three decades, developing and delivering < constantly advances its features and functional\n\nhttps://www.ibm.com/case-studies/us-open 2/12\n\n4/9125, 8:57 PM U.S. Open | IBM “The digital experience of the US Open is of enc and therefore to us,” says Kirsten Corio, Chi

In [20]:
#split the data in these documents into smaller chunks that can be processed by the LLM

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [21]:
#intialize embedding model

embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=credentials["project_id"],
)

In [22]:
# set up a vector store the documents and index them

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",
    embedding=embeddings,
)

In [23]:
# set up a retriever to get the documents from the vector store

retriever = vectorstore.as_retriever()


### Test prompts

In [24]:
# define a "tool" that the AI agent can use to get information from the vector store
# this is similar to a function that takes in a question and returns the answer
# the tool will use the retriever to get the relevant documents and then pass them to the LLM for processing

@tool
def get_IBM_US_Open_context(question: str):
    """Get context about IBM's involvement in the 2024 US Open Tennis Championship."""
    context = retriever.invoke(question)
    return context


tools = [get_IBM_US_Open_context] # list of tools that the agent can use

In [25]:
# set up system prompt for the agent (print its thought process, the tools that were used, and the final answer)

system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [26]:
# establish the human prompt (print the user input followed by the intermediate steps of the agent as part of the agent's "scratchpad")

human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

In [27]:
# establish the order of our newly defined prompts in the prompt template

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

In [28]:
#finalize our prompt template by adding the tool names, descriptions, and arguments using a partial prompt template
# So instead of needing to pass these values (tools and tool_names) every time you use the prompt, you partially apply them once using the `partial` method.
# This way, every time you use the prompt, it will automatically include the tool names and descriptions.

prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [29]:
# Set up agent memory to store past conversation history and context to help the agent remember what has been discussed in the past and improve its responses

memory = ConversationBufferMemory()


/var/folders/t1/wwm3r3cx5yz8nx7bf5x72w2w0000gn/T/ipykernel_38878/1271133669.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [30]:
# set up agent chain (scratchpad to format the log of the agent's thought process and actions taken, memory, prompt, and LLM)

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory
)

In [31]:
agent_executor.invoke({"input": "Where was the 2024 US Open Tennis Championship?"})




> Entering new AgentExecutor chain...


Question: Where was the 2024 US Open Tennis Championship?
Thought: I don't have the context about the location of the 2024 US Open Tennis Championship. I need to use the get_IBM_US_Open_context tool to find out.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "Where was the 2024 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'source': 'doc 4.pdf'}, page_content="4/9/25, 8:59 PM IBM and the USTA Serve Up New and Enhanced Generative Al Features for 2024 US Open Digital Platforms\n\nIBM and the USTA Serve Up New and Enhanced Generative AI Features for 2024 US Open Digital Platforms\n\nNew Match Report summaries offer fans detailed analysis for all 254 US Open main draw singles matches\n\nEnhanced version of AI Commentary returns to US Open digital platforms for on-demand highlights, along with fully redesigned IBM SlamTracker experience\n\nIBM and USTA Foundation announce new collaboration to provide AI p

{'input': 'Where was the 2024 US Open Tennis Championship?',
 'history': '',
 'output': 'The 2024 US Open Tennis Championship was held in Flushing, New York.'}

In [32]:
agent_executor.invoke(
    {"input": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"}
)



> Entering new AgentExecutor chain...


Thought: I need to get the context about IBM's involvement in the 2024 US Open Tennis Championship.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'source': 'doc 2.pdf'}, page_content='4/9/25, 8:58 PM IBM and the US Open\n\nHome / Sports / US Open\n\nIBM at the US Open\n\nThis is tennis...on AI. With the help of watsonx and Granite, IBM is infusing t US Open app with Al-generated insigt and content.\n\nWatch the video >\n\nhttps://www.ibm.com/sports/usopen 1/10\n\n4/9/25, 8:58 PM IBM and the US Open\n\nStay informed on all things AI\n\nDigital experience\n\nThe US Open App\n\nRead more\n\nhttps://www.ibm.com/sports/usopen 2/10\n\n4/9/25, 8:58 PM IBM and the US Open\n\nAl for business\n\nHow clients are using watsonx\n\nRead more \\\n\nThe US Open App\n\nFor more than 30 years, IBM has partnered witt experiences for mil

{'input': 'How did IBM use watsonx at the 2024 US Open Tennis Championship?',
 'history': 'Human: Where was the 2024 US Open Tennis Championship?\nAI: The 2024 US Open Tennis Championship was held in Flushing, New York.',
 'output': 'IBM used watsonx at the 2024 US Open Tennis Championship to provide AI-generated insights and content in the US Open app. This included spoken comment previews, match summaries, and enhanced AI commentary for on-demand highlights. Additionally, IBM and the USTA Foundation announced a collaboration to provide AI professional development resources to USTA Foundation students, teaching professionals, and the public.'}

In [33]:
agent_executor.invoke({"input": "What is the capital of France?"})




> Entering new AgentExecutor chain...


```
{
  "action": "Final Answer",
  "action_input": "The capital of France is Paris."
}
```
Observation

> Finished chain.


{'input': 'What is the capital of France?',
 'history': 'Human: Where was the 2024 US Open Tennis Championship?\nAI: The 2024 US Open Tennis Championship was held in Flushing, New York.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM used watsonx at the 2024 US Open Tennis Championship to provide AI-generated insights and content in the US Open app. This included spoken comment previews, match summaries, and enhanced AI commentary for on-demand highlights. Additionally, IBM and the USTA Foundation announced a collaboration to provide AI professional development resources to USTA Foundation students, teaching professionals, and the public.',
 'output': 'The capital of France is Paris.'}

In [34]:
agent_executor.invoke({"input": "Where was the 2026 US Open Tennis Championship?"})




> Entering new AgentExecutor chain...


Thought: The user is asking about the location of the 2026 US Open Tennis Championship. I do not have this information. I will use the "get_IBM_US_Open_context" tool to find out if there is any context about IBM's involvement in the 2026 US Open.

Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "What is IBM's involvement in the 2026 US Open Tennis Championship?"
}
```

Observation[Document(metadata={'source': 'doc 3.pdf'}, page_content='Although IBM has been providing technology support to USTA for the US Open for 29 years, the main emphasis in the past was on assuring a successful, seamless website and mobile app experience. This time, with only weeks to prepare, they needed make it an immersive virtual experience. Fortunately, because the IBM iX team has created digital content to support ESPN Fantasy Football and other sports events, it could draw on extensive virtual sports expertise.\n\nOpen Questions with Watson Dis

{'input': 'Where was the 2026 US Open Tennis Championship?',
 'history': 'Human: Where was the 2024 US Open Tennis Championship?\nAI: The 2024 US Open Tennis Championship was held in Flushing, New York.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM used watsonx at the 2024 US Open Tennis Championship to provide AI-generated insights and content in the US Open app. This included spoken comment previews, match summaries, and enhanced AI commentary for on-demand highlights. Additionally, IBM and the USTA Foundation announced a collaboration to provide AI professional development resources to USTA Foundation students, teaching professionals, and the public.\nHuman: What is the capital of France?\nAI: The capital of France is Paris.',
 'output': "The provided context does not mention the location of the 2026 US Open Tennis Championship. IBM has been providing technology support to USTA for the US Open for 29 years, focusing on creating immersive virtual e